## Working with Batches

The Batch API endpoint allows users to submit requests for asynchronous batch processing. We will process these requests within 24 hours. The details of each request will be read from a pre-uploaded file, and the responses will be written to an output file. You can query the batch object for status updates and results. Each model will be offered at 50% cost discount vs. the synchronous APIs. 



# Univeral Code Used for the Entire Notebook

Let's set up our libraries and client

In [8]:
# Import the OpenAI class from the openai module
from openai import OpenAI

# Import the time module to allow for time-related functions
import time


In [3]:
# Create a client we can use
client = OpenAI()

## Preparing the Batch File

Batches start with a .jsonl file where each line contains the details of an individual request to the API. For now, the available endpoints are /v1/chat/completions (Chat Completions API) and /v1/embeddings (Embeddings API). For a given input file, the parameters in each line's body field are the same as the parameters for the underlying endpoint. Each request must include a unique custom_id value, which you can use to reference results after completion. Here's an example of an input file with 2 requests. Note that each input file can only include requests to a single model.

NOTE: For some insane reason you are required to indicate the API endpoint here and in the batch creation later on. Presumably, OpenAI will one day allow hitting multiple different APIs in one batch request; but today is not that day. 

<br/>
Examples:

```
{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Give me three paragraphs on the penguin lifecycle."}],"max_tokens": 1000}}

{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Give me three paragraphs on penguin mating habits."}],"max_tokens": 1000}}

{"custom_id": "request-3", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Give me three paragraphs on penguin species differences."}],"max_tokens": 1000}}
```

### Uploading the File

After creating your batch file, you must upload it so that you can reference it correctly when kicking off batches. Upload your .jsonl file using the Files API.

In [4]:
# Create a file object for the bad batch input file and upload it using the OpenAI client
bad_batch_input_file = client.files.create(
    file=open("./artifacts/badbatchinput.jsonl", "rb"),  # Open the file in read-binary mode
    purpose="batch"  # Specify the purpose of the file
)

# Create a file object for the good batch input file and upload it using the OpenAI client
good_batch_input_file = client.files.create(
    file=open("./artifacts/goodbatchinput.jsonl", "rb"),  # Open the file in read-binary mode
    purpose="batch"  # Specify the purpose of the file
)

## Creating the Batch

Once you've successfully uploaded your input file, you can use the input File object's ID to create a batch. For now, the completion window can only be set to 24h. You can also provide custom metadata via an optional metadata parameter.

In [5]:
# Get the ID of the bad batch input file
bad_batch_input_file_id = bad_batch_input_file.id

# Create a batch job using the bad batch input file
bad_batch = client.batches.create(
    input_file_id=bad_batch_input_file_id,  # ID of the input file for the batch
    endpoint="/v1/chat/completions",  # API endpoint to use for the batch
    completion_window="24h",  # Time window for completion
    metadata={
        "description": "bad nightly penguin job"  # Metadata describing the batch job
    }
)

# Print the details of the bad batch job
print(bad_batch)
print("\n\n")

# Get the ID of the good batch input file
good_batch_input_file_id = good_batch_input_file.id

# Create a batch job using the good batch input file
good_batch = client.batches.create(
    input_file_id=good_batch_input_file_id,  # ID of the input file for the batch
    endpoint="/v1/chat/completions",  # API endpoint to use for the batch
    completion_window="24h",  # Time window for completion
    metadata={
        "description": "good nightly penguin job"  # Metadata describing the batch job
    }
)

# Print the details of the good batch job
print(good_batch)

Batch(id='batch_DT9jXFVLvP6UkeVZbzVBaEYg', completion_window='24h', created_at=1720349807, endpoint='/v1/chat/completions', input_file_id='file-UYzd9ehLSnVkQ6O6mUBixdXo', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720436207, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'bad nightly penguin job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))



Batch(id='batch_Maz2Rs4IBwiPHzP1O6yfyn1F', completion_window='24h', created_at=1720349808, endpoint='/v1/chat/completions', input_file_id='file-xwsJPwxz9OgOtkchtQt3uDxS', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720436208, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'good nightly penguin job'}, output_fil

## Checking Batch Status

The status of a given Batch object can be any of the following:

| STATUS      | DESCRIPTION                                                                  |
|-------------|------------------------------------------------------------------------------|
| validating  | the input file is being validated before the batch can begin                 |
| failed      | the input file has failed the validation process                             |
| in_progress | the input file was successfully validated and the batch is currently being run |
| finalizing  | the batch has completed and the results are being prepared                   |
| completed   | the batch has been completed and the results are ready                       |
| expired     | the batch was not able to be completed within the 24-hour time window        |
| cancelling  | the batch is being cancelled (may take up to 10 minutes)                     |
| cancelled   | the batch was cancelled                                                      |


In [6]:
# Retrieve and print the details of the bad batch job
print(client.batches.retrieve(bad_batch.id))
print("\n\n")  # Print newline characters for better readability

# Retrieve and print the details of the good batch job
print(client.batches.retrieve(good_batch.id))


Batch(id='batch_DT9jXFVLvP6UkeVZbzVBaEYg', completion_window='24h', created_at=1720349807, endpoint='/v1/chat/completions', input_file_id='file-UYzd9ehLSnVkQ6O6mUBixdXo', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720436207, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'bad nightly penguin job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))



Batch(id='batch_Maz2Rs4IBwiPHzP1O6yfyn1F', completion_window='24h', created_at=1720349808, endpoint='/v1/chat/completions', input_file_id='file-xwsJPwxz9OgOtkchtQt3uDxS', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720436208, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'good nightly penguin job'}, output_fil

## Listing the Batches

At any time, you can see all your batches. For users with many batches, you can use the limit and after parameters to paginate your results.

In [18]:
# Print a list of all batches
print(client.batches.list(limit=1))

# Convert the returned object to a list to get the length
batches = list(client.batches.list())

# Print the total number of batches
print("\nWe have " + str(len(batches)) + " batches\n")

print("========= ALL BATCHES ==========\n")
# Iterate over the batches and print their ids, statuses, and descriptions
for batch in batches:
    print(batch.id)  # Print the batch ID
    print(batch.status)  # Print the batch status
    print(batch.metadata.get("description"))  # Print the batch description from metadata
    print("\n\n")  # Print newline characters for better readability
    
print("========= JUST GOOD BATCHES ==========\n")
# Iterate over the good batches only and print their ids, statuses, and descriptions
for batch in batches:
    if batch.metadata.get("description", "").startswith("good"):
        print(batch.id)  # Print the batch ID
        print(batch.status)  # Print the batch status
        print(batch.metadata.get("description"))  # Print the batch description from metadata
        print("\n\n")  # Print newline characters for better readability


# Do a deep dive on these at https://github.com/openai/openai-python/blob/main/src/openai/resources/batches.py
# If you want to see the source code for the batches 



SyncCursorPage[Batch](data=[Batch(id='batch_Maz2Rs4IBwiPHzP1O6yfyn1F', completion_window='24h', created_at=1720349808, endpoint='/v1/chat/completions', input_file_id='file-xwsJPwxz9OgOtkchtQt3uDxS', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1720349816, error_file_id=None, errors=None, expired_at=None, expires_at=1720436208, failed_at=None, finalizing_at=1720349815, in_progress_at=1720349809, metadata={'description': 'good nightly penguin job'}, output_file_id='file-VpPFul3vJxByed5mL4qfvsDU', request_counts=BatchRequestCounts(completed=3, failed=0, total=3))], object='list', first_id='batch_Maz2Rs4IBwiPHzP1O6yfyn1F', last_id='batch_Maz2Rs4IBwiPHzP1O6yfyn1F', has_more=True)

We have 33 batches

========= ALL BATCHES ==========

batch_Maz2Rs4IBwiPHzP1O6yfyn1F
completed
good nightly penguin job



batch_DT9jXFVLvP6UkeVZbzVBaEYg
failed
bad nightly penguin job



batch_SSQ7yjhBWydbevJm6xkN6rjU
completed
good nightly penguin job



batch_b78mnTOiF

## Cancelling a Batch

If necessary, you can cancel an ongoing batch. The batch's status will change to cancelling until in-flight requests are complete (up to 10 minutes), after which the status will change to cancelled.

In [21]:
good_batch_input_file_id = good_batch_input_file.id

dead_batch_walking = client.batches.create(
    input_file_id=good_batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "good nightly penguin job"
    }
)

print(dead_batch_walking)
time.sleep(5)
print("\n\n")
print(client.batches.retrieve(dead_batch_walking.id))
print("\n\n")
print(client.batches.cancel(dead_batch_walking.id))
time.sleep(15)
print("\n\n")
print(client.batches.retrieve(dead_batch_walking.id))

Batch(id='batch_dXw9DBgnIRkSDImuhpOUsxo2', completion_window='24h', created_at=1720353424, endpoint='/v1/chat/completions', input_file_id='file-xwsJPwxz9OgOtkchtQt3uDxS', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720439824, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'good nightly penguin job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))



Batch(id='batch_dXw9DBgnIRkSDImuhpOUsxo2', completion_window='24h', created_at=1720353424, endpoint='/v1/chat/completions', input_file_id='file-xwsJPwxz9OgOtkchtQt3uDxS', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1720439824, failed_at=None, finalizing_at=None, in_progress_at=1720353424, metadata={'description': 'good nightly penguin job'}, ou

## Getting the Results

Once the batch is complete, you can download the output by making a request against the Files API via the output_file_id field from the Batch object and writing it to a file on your machine, in this case batch_output.jsonl

In [22]:

# Fetch the content of the file
content = client.files.content(client.batches.retrieve(good_batch.id).output_file_id)

content.write_to_file("./artifacts/goodbatchoutput.jsonl")
